In [6]:
import numpy as np

import xtrack as xt
import xpart as xp

#################################
# Load a line and build tracker #
#################################

line = xt.Line.from_json('line_and_particle.json')
line.particle_ref = xp.Particles(
                    mass0=xp.PROTON_MASS_EV, q0=1, energy0=7e12)
line.build_tracker()

#########
# Twiss #
#########

tw = line.twiss()

#!end-doc-part

# Test custom s locations
s_test = [2e3, 1e3, 3e3, 10e3]
twats = line.twiss(at_s = s_test)
for ii, ss in enumerate(s_test):
    assert np.isclose(twats['s'][ii], ss, rtol=0, atol=1e-14)
    i_prev = np.where(tw['s']<=ss)[0][-1]
    assert np.isclose(twats['alfx'][ii], np.interp(ss, tw['s'], tw['alfx']),
                      rtol=0, atol=1e-9)
    assert np.isclose(twats['alfy'][ii], np.interp(ss, tw['s'], tw['alfy']),
                     rtol=0, atol=1e-9)
    assert np.isclose(twats['dpx'][ii], np.interp(ss, tw['s'], tw['dpx']),
                      rtol=0, atol=1e-9)
    assert np.isclose(twats['dpy'][ii], np.interp(ss, tw['s'], tw['dpy']),
                      rtol=0, atol=1e-9)


twmb19r5 = tw.get_twiss_init(at_element='mb.b19l5.b1')

tw_part = line.twiss(ele_start='mb.b19l5.b1', ele_stop='mb.b19r5.b1',
                        twiss_init=twmb19r5)


import matplotlib.pyplot as plt

plt.close('all')

fig1 = plt.figure(1, figsize=(6.4, 4.8*1.5))
spbet = plt.subplot(3,1,1)
spco = plt.subplot(3,1,2, sharex=spbet)
spdisp = plt.subplot(3,1,3, sharex=spbet)

spbet.plot(tw['s'], tw['betx'])
spbet.plot(tw['s'], tw['bety'])

spco.plot(tw['s'], tw['x'])
spco.plot(tw['s'], tw['y'])

spdisp.plot(tw['s'], tw['dx'])
spdisp.plot(tw['s'], tw['dy'])

spbet.set_ylabel(r'$\beta_{x,y}$ [m]')
spco.set_ylabel(r'(Closed orbit)$_{x,y}$ [m]')
spdisp.set_ylabel(r'$D_{x,y}$ [m]')
spdisp.set_xlabel('s [m]')

fig1.suptitle(
    r'$q_x$ = ' f'{tw["qx"]:.5f}' r' $q_y$ = ' f'{tw["qy"]:.5f}' '\n'
    r"$Q'_x$ = " f'{tw["dqx"]:.2f}' r" $Q'_y$ = " f'{tw["dqy"]:.2f}'
    r' $\gamma_{tr}$ = '  f'{1/np.sqrt(tw["momentum_compaction_factor"]):.2f}'
)

fig1.subplots_adjust(left=.15, right=.92, hspace=.27)
plt.show()

Done loading line from dict.           
Found suitable prebuilt kernel `default_only_xtrack`.


ValueError: Invalid one-turn map: No coordinates respond to variations of x

In [7]:
import json
import numpy as np

import xobjects as xo
import xpart as xp
import xtrack as xt
import xfields as xf

############
# Settings #
############

fname_line = ('line_no_spacecharge_and_particle.json')

bunch_intensity = 1e11/3 # Need short bunch to avoid bucket non-linearity
                         # to compare frozen/quasi-frozen and PIC
sigma_z = 22.5e-2/3
nemitt_x=2.5e-6
nemitt_y=2.5e-6
n_part=int(1e6)
num_turns=32

num_spacecharge_interactions = 540
tol_spacecharge_position = 1e-2

# Available modes: frozen/quasi-frozen/pic
mode = 'pic'

####################
# Choose a context #
####################

context = xo.ContextCpu()
#context = xo.ContextCupy()
#context = xo.ContextPyopencl('0.0')

print(context)

#############
# Load line #
#############

with open(fname_line, 'r') as fid:
     input_data = json.load(fid)
line = xt.Line.from_dict(input_data['line'])
particle_ref = xp.Particles.from_dict(input_data['particle'])

#############################################
# Install spacecharge interactions (frozen) #
#############################################

lprofile = xf.LongitudinalProfileQGaussian(
        number_of_particles=bunch_intensity,
        sigma_z=sigma_z,
        z0=0.,
        q_parameter=1.)

xf.install_spacecharge_frozen(line=line,
                   particle_ref=particle_ref,
                   longitudinal_profile=lprofile,
                   nemitt_x=nemitt_x, nemitt_y=nemitt_y,
                   sigma_z=sigma_z,
                   num_spacecharge_interactions=num_spacecharge_interactions,
                   tol_spacecharge_position=tol_spacecharge_position)

#################################
# Switch to PIC or quasi-frozen #
#################################

if mode == 'frozen':
    pass # Already configured in line
elif mode == 'quasi-frozen':
    xf.replace_spacecharge_with_quasi_frozen(
                                    line,
                                    update_mean_x_on_track=True,
                                    update_mean_y_on_track=True)
elif mode == 'pic':
    pic_collection, all_pics = xf.replace_spacecharge_with_PIC(
        _context=context, line=line,
        n_sigmas_range_pic_x=8,
        n_sigmas_range_pic_y=8,
        nx_grid=256, ny_grid=256, nz_grid=100,
        n_lims_x=7, n_lims_y=3,
        z_range=(-3*sigma_z, 3*sigma_z))
else:
    raise ValueError(f'Invalid mode: {mode}')


#################
# Build Tracker #
#################

line.build_tracker(_context=context)
line_sc_off = line.filter_elements(exclude_types_starting_with='SpaceCh')

######################
# Generate particles #
######################

# (we choose to match the distribution without accounting for spacecharge)
particles = xp.generate_matched_gaussian_bunch(_context=context,
         num_particles=n_part, total_intensity_particles=bunch_intensity,
         nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z=sigma_z,
         particle_ref=particle_ref, line=line_sc_off)

#########
# Track #
#########

line.track(particles, num_turns=3)

ContextCpu


Done loading line from dict.           
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
*** Maximum RMS bunch length 0.2357884536170542m.


c:\Users\bengl\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\integrate\_quadpack_py.py:1225: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


... distance to target bunch length: -7.5000e-02
... distance to target bunch length: 1.5427e-01
... distance to target bunch length: 9.1211e-02
... distance to target bunch length: -3.0747e-03
... distance to target bunch length: 3.0774e-05
... distance to target bunch length: -1.1076e-07
... distance to target bunch length: 2.8554e-07
--> Bunch length: 0.07499988923724311
--> Emittance: 0.03748900029855025
Creating PIC (2, 0)
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Creating PIC (1, 1)
Creating PIC (0, 2)
Creating PIC (3, 0)
Creating PIC (2, 1)
Creating PIC (1, 2)
Creating PIC (3, 1)
Creating PIC (5, 0)
Creating PIC (2, 2)
Creating PIC (4, 1)
Creating PIC (4, 0)
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
*** Maximum RMS bunch length 0.23578845361705414m.
... distance to target bunch length: -7.5000e-02
... distance to target bunch length: 1.5427e-01
... distance to target bunch length: 9.1211e-02
... distance to target bunch length: -

KeyboardInterrupt: 